In [1]:
! pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai faiss-cpu

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("D:\soft\workspace\pythonworkspace\LangChainStudy\documentstore\state_of_the_union.txt")
documents = loader.load()

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OllamaEmbeddings()
db = FAISS.from_documents(texts, embeddings)

In [6]:
retriever = db.as_retriever()

In [7]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_state_of_union",
    "Searches and returns excerpts from the 2022 State of the Union.",
)
tools = [tool]

In [8]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0,base_url="http://localhost:11434/v1",model="llama2")

In [10]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [11]:
result = agent_executor.invoke({"input": "hi, im bob"})

In [12]:
result["output"]

"Hello Bob! It's nice to meet you. Is there something I can help you with or would you like me to assist you in any way? Please let me know how I can be of service."

In [13]:
result = agent_executor.invoke(
    {
        "input": "what did the president say about ketanji brown jackson in the most recent state of the union?"
    }
)

In [14]:
result["output"]

" I'm just an AI, I don't have access to real-time information or current events, including the President's statements in the most recent State of the Union address. The State of the Union is a annual address delivered by the President of the United States to Congress, typically in February or March of each year. It provides an update on the state of the union and outlines the President's legislative agenda for the upcoming year.\n\nHowever, I can tell you that Ketanji Brown Jackson is a current Supreme Court Justice appointed by President Joe Biden in 2020. She is the first African American woman to serve on the Supreme Court. Prior to her appointment to the Supreme Court, Justice Jackson served as a federal judge on the United States District Court for the Southern District of Florida.\n\nIf you have any other questions or concerns, feel free to ask!"

In [15]:
result = agent_executor.invoke(
    {"input": "how long ago did the president nominate ketanji brown jackson?"}
)

In [16]:
result["output"]

"\nI'm just an AI, I don't have access to real-time information or current events, so I cannot provide you with the exact date when President Joe Biden nominated Ketanji Brown Jackson to the United States Supreme Court. However, I can tell you that President Biden nominated Judge Ketanji Brown Jackson on February 24, 2022, to fill the seat left vacant by the retirement of Justice Stephen G. Breyer."